In [86]:
import pandas as pd
import numpy as np

import requests, bs4
from bs4 import BeautifulSoup as bs

import time, random

import csv

import seaborn as sns

In [87]:
# Make some room to see stuff (i.e. drop display limits on Pandas rows & cols - be careful w/ big df's!)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 1) Combine all csv files generated during web scraping and clean

In [88]:
files = ['home_details_WestValley_1.csv', 'home_details_WestValley_2.csv', 'home_details_SouthCentralValley.csv',
         'home_details_NorthValley.csv', 'home_details_NorthCentralValley.csv', 'home_details_SouthEastBottom.csv',
         'home_details_SouthEastBottomUpOne.csv', 'home_details_SouthEastBottomUpTwo.csv',
         'home_details_SouthEastBottomUpThree.csv', 'home_details_SouthEastCenterBottom.csv',
         'home_details_SouthEastCenterCenter.csv', 'home_details_SouthEastCenterTop.csv',
         'home_details_NorthEastCenterBottom.csv', 'home_details_NorthEastCenterCenter.csv',
         'home_details_NorthEastUpperBottom.csv', 'home_details_NorthEastUpperCenter.csv',
         'home_details_NorthEastUpperTop.csv', 'home_details_EasternEdge.csv']

for n, file in enumerate(files):
    if n == 0:
        df = pd.read_csv('home_details/' + file)
    else:
        df_temp = pd.read_csv('home_details/' +file)
        df = pd.concat([df, df_temp], axis=0)
        df.reset_index(drop=True, inplace=True)
        
df.columns = [col.strip() for col in df.columns]

In [89]:
df = df[~df.isna().any(axis=1)]
df = df.drop_duplicates()
df.shape

(1313, 11)

In [90]:
df['stories'].value_counts()

1    864
2    185
—    162
1     93
2      7
3      2
Name: stories, dtype: int64

### Drop the rows with '-' for the 'stories' column

In [91]:
df.head()

,sold_price,beds,baths,hm_sqft,lot_size,yr_built,zipcode,pool,garage,stories,schools
0,970000,4,3.0,1933,"6,999 Sq. Ft.",1960,91364,True,True,1,6.4
1,900000,3,2.0,1519,0.27 Acres,1974,91304,True,True,1,6.4
2,887000,4,3.0,1845,"2,555 Sq. Ft.",2020,91307,True,True,—,6.4
4,830000,3,2.0,1595,"10,560 Sq. Ft.",1960,91303,True,True,1,4.8
5,820000,3,2.5,1364,"7,253 Sq. Ft.",1959,91307,False,False,1,7.0


In [92]:
null_val = df['stories'][2]
null_val

'—'

In [93]:
df = df[df['stories'] != null_val]
df.reset_index(drop=True, inplace=True)
df.shape

(1151, 11)

In [94]:
df['stories'].value_counts()

1    864
2    185
1     93
2      7
3      2
Name: stories, dtype: int64

### Corrupted row (parsing error) - pretty obvious how to fix it

In [95]:
df.loc[912]

sold_price    AveNorthridge
beds                  91325
baths              995000.0
hm_sqft       1Bed1Bath1124
lot_size         0.56 Acres
yr_built               1954
zipcode                 Oak
pool                  False
garage                False
stories                   1
schools                 5.2
Name: 912, dtype: object

In [96]:
df.loc[912, 'sold_price'] = '995000.0'
df.loc[912, 'beds'] = '1'
df.loc[912, 'baths'] = '1'
df.loc[912, 'hm_sqft'] = '1124'
df.loc[912, 'zipcode'] = '91325'

In [97]:
df.loc[912]

sold_price      995000.0
beds                   1
baths                  1
hm_sqft             1124
lot_size      0.56 Acres
yr_built            1954
zipcode            91325
pool               False
garage             False
stories                1
schools              5.2
Name: 912, dtype: object

### More corrupted rows (parsing errors) - pretty obvious how to fix them

In [98]:
for row in df.itertuples():
    if 'Hills' in row:
        print(row)

Pandas(Index=159, sold_price=91364, beds=902000, baths=3.0, hm_sqft='1Bath1014', lot_size='6,249 Sq. Ft.', yr_built=1949, zipcode='Hills', pool=False, garage=False, stories='1', schools=6.4)
Pandas(Index=246, sold_price=91364, beds=900000, baths=2.0, hm_sqft='1Bath1219', lot_size='5,287 Sq. Ft.', yr_built=1953, zipcode='Hills', pool=False, garage=False, stories='1', schools=6.4)
Pandas(Index=258, sold_price=91367, beds=740000, baths=3.0, hm_sqft='1Bath1069', lot_size='7,335 Sq. Ft.', yr_built=1952, zipcode='Hills', pool=False, garage=False, stories='1', schools=6.8)
Pandas(Index=431, sold_price='91364', beds='790000', baths='3', hm_sqft='1Bath1101', lot_size='9,329 Sq. Ft.', yr_built='1954', zipcode='Hills', pool=False, garage=False, stories='1', schools=6.0)
Pandas(Index=473, sold_price='Hills', beds='91364', baths='753000', hm_sqft='1Bed1Bath564', lot_size='5,112 Sq. Ft.', yr_built='1952', zipcode='DrWoodland', pool=False, garage=False, stories='1', schools=6.0)
Pandas(Index=496, sol

In [99]:
idx = 159
df.loc[idx, 'sold_price'] = '902000'
df.loc[idx, 'beds'] = '3'
df.loc[idx, 'baths'] = '1'
df.loc[idx, 'hm_sqft'] = '1014'
df.loc[idx, 'zipcode'] = '91364'

In [100]:
df.loc[idx]

sold_price           902000
beds                      3
baths                     1
hm_sqft                1014
lot_size      6,249 Sq. Ft.
yr_built               1949
zipcode               91364
pool                  False
garage                False
stories                   1
schools                 6.4
Name: 159, dtype: object

In [101]:
idx = 246
df.loc[idx, 'sold_price'] = '900000'
df.loc[idx, 'beds'] = '2'
df.loc[idx, 'baths'] = '1'
df.loc[idx, 'hm_sqft'] = '1219'
df.loc[idx, 'zipcode'] = '91364'

In [102]:
df.loc[idx]

sold_price           900000
beds                      2
baths                     1
hm_sqft                1219
lot_size      5,287 Sq. Ft.
yr_built               1953
zipcode               91364
pool                  False
garage                False
stories                   1
schools                 6.4
Name: 246, dtype: object

In [103]:
idx = 258
df.loc[idx, 'sold_price'] = '740000'
df.loc[idx, 'beds'] = '3'
df.loc[idx, 'baths'] = '1'
df.loc[idx, 'hm_sqft'] = '1069'
df.loc[idx, 'zipcode'] = '91367'

In [104]:
df.loc[idx]

sold_price           740000
beds                      3
baths                     1
hm_sqft                1069
lot_size      7,335 Sq. Ft.
yr_built               1952
zipcode               91367
pool                  False
garage                False
stories                   1
schools                 6.8
Name: 258, dtype: object

In [105]:
idx = 431
df.loc[idx, 'sold_price'] = '790000'
df.loc[idx, 'beds'] = '3'
df.loc[idx, 'baths'] = '1'
df.loc[idx, 'hm_sqft'] = '1101'
df.loc[idx, 'zipcode'] = '91364'

In [106]:
df.loc[idx]

sold_price           790000
beds                      3
baths                     1
hm_sqft                1101
lot_size      9,329 Sq. Ft.
yr_built               1954
zipcode               91364
pool                  False
garage                False
stories                   1
schools                 6.0
Name: 431, dtype: object

In [107]:
idx = 473
df.loc[idx, 'sold_price'] = '753000'
df.loc[idx, 'beds'] = '1'
df.loc[idx, 'baths'] = '1'
df.loc[idx, 'hm_sqft'] = '564'
df.loc[idx, 'zipcode'] = '91364'

In [108]:
df.loc[idx]

sold_price           753000
beds                      1
baths                     1
hm_sqft                 564
lot_size      5,112 Sq. Ft.
yr_built               1952
zipcode               91364
pool                  False
garage                False
stories                   1
schools                 6.0
Name: 473, dtype: object

In [109]:
idx = 496
df.loc[idx, 'sold_price'] = '902000'
df.loc[idx, 'beds'] = '3'
df.loc[idx, 'baths'] = '1'
df.loc[idx, 'hm_sqft'] = '1014'
df.loc[idx, 'zipcode'] = '91364'

In [110]:
# This is a duplicate of the first of these we dealt with; the parsing error
#   must have fooled .drop_duplicates(); will have to drop it

df.loc[idx]

sold_price           902000
beds                      3
baths                     1
hm_sqft                1014
lot_size      6,249 Sq. Ft.
yr_built               1949
zipcode               91364
pool                  False
garage                False
stories                   1
schools                 6.4
Name: 496, dtype: object

In [111]:
print(df.shape)
df = df[df.index != idx]
print(df.shape)

(1151, 11)
(1150, 11)


In [112]:
idx = 562
df.loc[idx, 'sold_price'] = '900000'
df.loc[idx, 'beds'] = '2'
df.loc[idx, 'baths'] = '1'
df.loc[idx, 'hm_sqft'] = '1219'
df.loc[idx, 'zipcode'] = '91364'

In [113]:
# This looks like another duplicate! Will drop it, too

df.loc[idx]

sold_price           900000
beds                      2
baths                     1
hm_sqft                1219
lot_size      5,287 Sq. Ft.
yr_built               1953
zipcode               91364
pool                  False
garage                False
stories                   1
schools                 6.4
Name: 562, dtype: object

In [114]:
print(df.shape)
df = df[df.index != idx]
print(df.shape)

(1150, 11)
(1149, 11)


In [115]:
idx = 682
df.loc[idx, 'sold_price'] = '815000'
df.loc[idx, 'beds'] = '3'
df.loc[idx, 'baths'] = '1'
df.loc[idx, 'hm_sqft'] = '1147'
df.loc[idx, 'zipcode'] = '91344'

In [116]:
df.loc[idx]

sold_price           815000
beds                      3
baths                     1
hm_sqft                1147
lot_size      7,148 Sq. Ft.
yr_built               1947
zipcode               91344
pool                  False
garage                False
stories                   1
schools                 5.0
Name: 682, dtype: object

In [117]:
# Re-index

df.reset_index(drop=True, inplace=True)

### Another anomaly: Lot Sale (no house) - must drop this row

In [118]:
df[df['beds'] == null_val]

,sold_price,beds,baths,hm_sqft,lot_size,yr_built,zipcode,pool,garage,stories,schools
559,1350000,—,—,—,0.40 Acres,1956,91364,False,False,1,6.0


In [119]:
print(df.shape)
df = df[df.index != 559]
print(df.shape)

(1149, 11)
(1148, 11)


### More anomalies (another easily fixable parsing error)

In [120]:
df[df['beds'].astype('int') > 20]

,sold_price,beds,baths,hm_sqft,lot_size,yr_built,zipcode,pool,garage,stories,schools
33,91303,615000,3.0,1Bath945,"4,961 Sq. Ft.",1951,Park,False,True,1,4.2
117,91303,701000,3.0,1Bath1268,"6,752 Sq. Ft.",1954,Park,True,True,1,4.8
251,91303,675000,2.0,1Bath1045,"6,751 Sq. Ft.",1951,Park,False,False,1,4.8
293,91335,704000,3,1Bath1148,"6,000 Sq. Ft.",1951,StReseda,False,False,1,5.2
294,91306,640000,3,1Bath1074,"6,752 Sq. Ft.",1949,AveWinnetka,False,False,1,5.0
298,91335,665000,2,1Bath885,"6,755 Sq. Ft.",1954,AveReseda,False,False,1,5.0
323,91335,725000,3,1Bath1073,"6,239 Sq. Ft.",1951,AveReseda,False,False,1,4.6
324,91306,700000,2,1Bath1003,0.40 Acres,1949,StWinnetka,False,False,1,4.8
342,91303,700000,3,1Bath1000,"5,006 Sq. Ft.",1956,Park,False,False,1,4.2
345,91306,661000,2,1Bath799,"7,067 Sq. Ft.",1951,AveWinnetka,False,False,1,5.0


In [121]:
# Same error for all of these; just move current 'sold_price' to 'zipcode', 'beds' to 'sold_price',
#   'baths' to 'beds', and split out char string in 'hm_sqft' appropriately beteween that column
#    and 'baths'

for row in df[df['beds'].astype('int') > 20].itertuples():
    df.loc[row[0], 'zipcode'] = df.loc[row[0], 'sold_price']
    df.loc[row[0], 'sold_price'] = df.loc[row[0], 'beds']
    df.loc[row[0], 'beds'] = df.loc[row[0], 'baths']
    df.loc[row[0], 'baths'] = df.loc[row[0], 'hm_sqft'].split('Bath')[0]
    df.loc[row[0], 'hm_sqft'] = df.loc[row[0], 'hm_sqft'].split('Bath')[1]

In [122]:
df[df['beds'].astype('int') > 20]

,sold_price,beds,baths,hm_sqft,lot_size,yr_built,zipcode,pool,garage,stories,schools


### Another anomaly (lazy realtor not filling in the info - found it by looking the property up)

In [123]:
df.loc[674]

sold_price     1137500.0
beds                   5
baths                  —
hm_sqft             2437
lot_size      0.28 Acres
yr_built            1965
zipcode            91326
pool               False
garage             False
stories                2
schools         6.166667
Name: 674, dtype: object

In [124]:
df.loc[674, 'baths'] = 3.0
df.loc[674]

sold_price     1137500.0
beds                   5
baths                3.0
hm_sqft             2437
lot_size      0.28 Acres
yr_built            1965
zipcode            91326
pool               False
garage             False
stories                2
schools         6.166667
Name: 674, dtype: object

### Drop any decimal points that might appear in columns destined to become integer

In [125]:
df['sold_price'] = [str(val).split('.')[0] for val in df['sold_price']]
df['beds'] = [str(val).split('.')[0] for val in df['beds']]
df['hm_sqft'] = [str(val).split('.')[0] for val in df['hm_sqft']]
df['yr_built'] = [str(val).split('.')[0] for val in df['yr_built']]
df['zipcode'] = [str(val).split('.')[0] for val in df['zipcode']]
df['stories'] = [str(val).split('.')[0] for val in df['stories']]

### Set the column types appropriatedly

In [126]:
df.dtypes

sold_price     object
beds           object
baths          object
hm_sqft        object
lot_size       object
yr_built       object
zipcode        object
pool             bool
garage           bool
stories        object
schools       float64
dtype: object

In [127]:
df['sold_price'] = df['sold_price'].astype('int')
df['beds'] = df['beds'].astype('int')
df['baths'] = df['baths'].astype('float')
df['hm_sqft'] = df['hm_sqft'].astype('int')
df['yr_built'] = df['yr_built'].astype('int')
df['zipcode'] = df['zipcode'].astype('int')
df['stories'] = df['stories'].astype('int')

In [128]:
df.dtypes

sold_price      int64
beds            int64
baths         float64
hm_sqft         int64
lot_size       object
yr_built        int64
zipcode         int64
pool             bool
garage           bool
stories         int64
schools       float64
dtype: object

### Special handling for 'lot_size' column: some values are in square feet, others in acres; need to fix this

In [129]:
df['lot_size'].head()

0     6,999 Sq. Ft.
1        0.27 Acres
2    10,560 Sq. Ft.
3     7,253 Sq. Ft.
4     9,347 Sq. Ft.
Name: lot_size, dtype: object

In [130]:
# 1 acre = 43560 square feet
# get rid of units and commas, convert acres to square feet where appropriate

df['lot_size'] = [str(int(float(val.split()[0])*43560)) if len(val.split()) == 2 \
                  else val.replace(' Sq. Ft.', '').replace(',', '') \
                  for val in df['lot_size'] ]

In [131]:
df['lot_size'] = df['lot_size'].astype('int')

In [132]:
df.dtypes

sold_price      int64
beds            int64
baths         float64
hm_sqft         int64
lot_size        int64
yr_built        int64
zipcode         int64
pool             bool
garage           bool
stories         int64
schools       float64
dtype: object

### Check for and drop any remaining duplicates

In [133]:
print(df.shape)
df = df.drop_duplicates()
print(df.shape)

(1148, 11)
(1011, 11)


### Let's Do Some Quick Summarization

In [134]:
df.describe()

,sold_price,beds,baths,hm_sqft,lot_size,yr_built,zipcode,stories,schools
count,1.011000e+03,1011.000000,1011.000000,1011.000000,1011.000000,1011.000000,1011.000000,1011.000000,1011.000000
mean,1.126206e+06,3.633037,2.544510,2081.720079,10828.724036,1960.297725,91341.634026,1.178042,5.738731
std,5.022851e+05,0.844819,0.924634,886.574023,15533.545033,11.616964,31.162630,0.387876,0.674897
min,4.000000e+05,1.000000,1.000000,552.000000,2000.000000,1920.000000,91302.000000,1.000000,4.200000
25%,8.400000e+05,3.000000,2.000000,1471.000000,7083.000000,1954.000000,91316.000000,1.000000,5.200000
50%,9.650000e+05,4.000000,2.000000,1873.000000,8217.000000,1958.000000,91343.000000,1.000000,5.666667
75%,1.260000e+06,4.000000,3.000000,2447.000000,11761.000000,1964.000000,91364.000000,1.000000,6.166667
max,5.500000e+06,8.000000,8.000000,8189.000000,475675.000000,2021.000000,91436.000000,3.000000,7.666667


# 2) Now bring in the additional csv data summaries downloaded from web

In [135]:
files = ['sup_2021-08-27-10-35-17_NorthCentralValley.csv', 'sup_2021-08-27-07-09-00_NorthValley.csv',
         'sup_2021-08-27-12-21-36_SouthCentralValley.csv', 'sup_2021-08-27-17-26-48_WestValley_Updated.csv',
         'sup_2021-08-27-18-27-26_SouthEastBottom.csv', 'sup_2021-08-27-19-25-38_SouthEastBottomUpOne.csv',
         'sup_2021-08-27-19-37-41_SouthEastBottomUpTwo.csv', 'sup_2021-08-27-19-52-02_SouthEastBottomUpThree.csv',
         'sup_2021-08-27-20-24-21_SouthEastCenterBottom.csv', 'sup_2021-08-28-15-23-04_SouthEastCenterCenter.csv',
         'sup_2021-08-28-15-43-33_SouthEastCenterTop.csv', 'sup_2021-08-28-15-59-43_NorthEastCenterBottom.csv',
         'sup_2021-08-28-16-33-14_NorthEastCenterCenter.csv', 'sup_2021-08-28-16-49-23_ NorthEastUpperBottom.csv',
         'sup_2021-08-28-17-06-30_NorthEastUpperCenter.csv', 'sup_2021-08-28-17-18-35_NorthEastUpperTop.csv',
         'sup_2021-08-28-23-42-30_EasternEdge.csv', 'sup_2021-08-29-16-22-49_91326.csv',
         'sup_2021-08-29-16-19-12_91364.csv', 'sup_2021-08-29-16-16-45_91307.csv',
         'sup_2021-08-29-16-14-17_91303.csv', 'sup_2021-08-29-16-09-53_91367_2.csv',
         'sup_2021-08-29-16-07-38_91304.csv', 'sup_2021-08-29-15-45-14_91367.csv',
         'sup_2021-08-29-16-32-27_91367_2.csv']

for n, file in enumerate(files):
    if n == 0:
        sup = pd.read_csv('supplemental_data/' + file)
    else:
        sup = pd.concat([sup, pd.read_csv('supplemental_data/' + file)])
        
sup.reset_index(drop=True, inplace=True)

In [136]:
sup.shape

(1506, 27)

In [137]:
sup = sup.drop(columns=['SALE TYPE', 'PROPERTY TYPE', 'BATHS', 'CITY', 'STATE OR PROVINCE', 'LOCATION', 
                        'LOT SIZE', 'NEXT OPEN HOUSE START TIME', 'NEXT OPEN HOUSE END TIME', 'SOURCE', 
                        'MLS#', 'FAVORITE', 'INTERESTED', 'STATUS', '$/SQUARE FEET'])

In [138]:
sup.head()

,SOLD DATE,ADDRESS,ZIP OR POSTAL CODE,PRICE,BEDS,SQUARE FEET,YEAR BUILT,DAYS ON MARKET,HOA/MONTH,URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),LATITUDE,LONGITUDE
0,July-22-2021,7937 N Keer Dr,91335.0,850000,5.0,2141.0,2016.0,36.0,41.0,http://www.redfin.com/CA/Reseda/7937-N-Keer-Dr...,34.215045,-118.535042
1,June-15-2021,7417 Delco Ave,91306.0,949000,5.0,2417.0,1979.0,73.0,NaN,http://www.redfin.com/CA/Canoga-Park/7417-Delc...,34.205416,-118.578453
2,August-20-2021,20541 Hart St,91306.0,730000,3.0,1212.0,1954.0,7.0,NaN,http://www.redfin.com/CA/Canoga-Park/20541-Har...,34.197026,-118.581189
3,June-14-2021,18043 Sunburst St,91325.0,1300000,4.0,2809.0,1957.0,74.0,NaN,http://www.redfin.com/CA/Northridge/18043-Sunb...,34.234288,-118.526605
4,June-1-2021,19508 Leadwell St,91335.0,635000,3.0,1097.0,1947.0,87.0,NaN,http://www.redfin.com/CA/Reseda/19508-Leadwell...,34.203339,-118.558299


In [139]:
sup.dtypes

SOLD DATE                                                                                      object
ADDRESS                                                                                        object
ZIP OR POSTAL CODE                                                                            float64
PRICE                                                                                           int64
BEDS                                                                                          float64
SQUARE FEET                                                                                   float64
YEAR BUILT                                                                                    float64
DAYS ON MARKET                                                                                float64
HOA/MONTH                                                                                     float64
URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON 

### Get the columns we want to merge on with df into the same format

In [140]:
# Drop rows with NaN in any of the columns we want to merge on

sup = sup[~sup[['ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'SQUARE FEET', 'YEAR BUILT']].isna().any(axis=1)]

In [141]:
sup['ZIP OR POSTAL CODE'] = sup['ZIP OR POSTAL CODE'].astype('int')
sup['PRICE'] = sup['PRICE'].astype('int')
sup['BEDS'] = sup['BEDS'].astype('int')
sup['SQUARE FEET'] = sup['SQUARE FEET'].astype('int')
sup['YEAR BUILT'] = sup['YEAR BUILT'].astype('int')

In [142]:
# Rename the merge columns of sup to make life easier (won't merge on 'BATHS' since lazy realtors
#    sometimes round the fractions to whole numbers)
sup.rename({'ZIP OR POSTAL CODE': 'zipcode',
            'PRICE': 'sold_price',
            'BEDS': 'beds',
            'SQUARE FEET': 'hm_sqft',
            'YEAR BUILT': 'yr_built'}, axis=1, inplace=True)
sup.head()

,SOLD DATE,ADDRESS,zipcode,sold_price,beds,hm_sqft,yr_built,DAYS ON MARKET,HOA/MONTH,URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),LATITUDE,LONGITUDE
0,July-22-2021,7937 N Keer Dr,91335,850000,5,2141,2016,36.0,41.0,http://www.redfin.com/CA/Reseda/7937-N-Keer-Dr...,34.215045,-118.535042
1,June-15-2021,7417 Delco Ave,91306,949000,5,2417,1979,73.0,NaN,http://www.redfin.com/CA/Canoga-Park/7417-Delc...,34.205416,-118.578453
2,August-20-2021,20541 Hart St,91306,730000,3,1212,1954,7.0,NaN,http://www.redfin.com/CA/Canoga-Park/20541-Har...,34.197026,-118.581189
3,June-14-2021,18043 Sunburst St,91325,1300000,4,2809,1957,74.0,NaN,http://www.redfin.com/CA/Northridge/18043-Sunb...,34.234288,-118.526605
4,June-1-2021,19508 Leadwell St,91335,635000,3,1097,1947,87.0,NaN,http://www.redfin.com/CA/Reseda/19508-Leadwell...,34.203339,-118.558299


In [143]:
# Drop duplicates
print(sup.shape)
sup = sup.drop_duplicates(subset=['zipcode', 'sold_price', 'beds', 'hm_sqft', 'yr_built'])
print(sup.shape)

(1500, 12)
(1394, 12)


In [144]:
merged = df.merge(sup, 
                  how='left',
                  on=['zipcode', 'sold_price', 'beds', 'hm_sqft', 'yr_built'])

In [145]:
merged.drop_duplicates().shape

(1011, 18)

In [147]:
df.shape

(1011, 11)

### Most rows have 'HOA/MONTH' set to NaN (i.e. no HOA applies) - let's change that to 0

In [148]:
merged.loc[merged['HOA/MONTH'].isna(), 'HOA/MONTH'] = 0

In [149]:
# Make sure no NaNs

merged[merged.isna().any(axis=1)]

,sold_price,beds,baths,hm_sqft,lot_size,yr_built,zipcode,pool,garage,stories,schools,SOLD DATE,ADDRESS,DAYS ON MARKET,HOA/MONTH,URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),LATITUDE,LONGITUDE


In [150]:
# Also make sure no duplicates and that indices are contiguous

print(merged.shape)
merged = merged.drop_duplicates()
merged.reset_index(drop=True, inplace=True)
print(merged.shape)

(1011, 18)
(1011, 18)


In [151]:
# Rename some cols for consistency

merged.rename({'SOLD DATE': 'sold_date',
               'DAYS ON MARKET': 'days_on_mkt',
               'HOA/MONTH': 'hoa',
               'LATITUDE': 'lat',
               'LONGITUDE': 'lon'},
               axis=1,
               inplace=True)

In [152]:
merged.rename(
    columns={
        'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)': 'url'},
    inplace=True)

In [153]:
merged.columns = [col.lower() for col in merged.columns]

In [154]:
merged['pool'] = merged['pool'].astype('int')
merged['garage'] = merged['garage'].astype('int')
merged['days_on_mkt'] = merged['days_on_mkt'].astype('int')
merged['hoa'] = merged['hoa'].astype('int')

In [155]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1011 entries, 0 to 1010
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sold_price   1011 non-null   int64  
 1   beds         1011 non-null   int64  
 2   baths        1011 non-null   float64
 3   hm_sqft      1011 non-null   int64  
 4   lot_size     1011 non-null   int64  
 5   yr_built     1011 non-null   int64  
 6   zipcode      1011 non-null   int64  
 7   pool         1011 non-null   int64  
 8   garage       1011 non-null   int64  
 9   stories      1011 non-null   int64  
 10  schools      1011 non-null   float64
 11  sold_date    1011 non-null   object 
 12  address      1011 non-null   object 
 13  days_on_mkt  1011 non-null   int64  
 14  hoa          1011 non-null   int64  
 15  url          1011 non-null   object 
 16  lat          1011 non-null   float64
 17  lon          1011 non-null   float64
dtypes: float64(4), int64(11), object(3)
memory usage

### Write this dataframe to csv

In [156]:
merged.to_csv('home_sales_dataset.csv', index=False)